We will also use the previously created train and validation files.

In [1]:
# import the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from scipy import ndimage
from sklearn.model_selection import train_test_split

2024-06-17 18:05:30.267125: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-17 18:05:31.559389: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 18:05:31.559465: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 18:05:31.710496: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-17 18:05:31.981894: I tensorflow/core/platform/cpu_feature_guar

In [2]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
sub_window_and_count_path_train = "../../S_lab_TasselNet/Method_2/final_train_sub_windows_and_counts"
sub_window_and_count_path_valid = "../../S_lab_TasselNet/Method_2/final_valid_sub_windows_and_counts"

In [4]:
%%time
# load the data

# subwindow data
X_train = np.load(os.path.join(sub_window_and_count_path_train, 'train_sub_windows.npy'))
print(X_train.shape)

# sub counts data
y_train = np.load(os.path.join(sub_window_and_count_path_train,'train_sub_counts.npy'))
print(y_train.shape)

(1572864, 32, 32, 3)
(1572864,)
CPU times: user 99 ms, sys: 29.7 s, total: 29.8 s
Wall time: 1min 20s


In [5]:
%%time
# load validation data
# sub windows
X_valid = np.load(os.path.join(sub_window_and_count_path_valid,'valid_sub_windows.npy'))
print(X_valid.shape)

# sub counts data
y_valid = np.load(os.path.join(sub_window_and_count_path_valid,'valid_sub_counts.npy'))
print(y_valid.shape)

(393216, 32, 32, 3)
(393216,)
CPU times: user 4.39 ms, sys: 7.64 s, total: 7.64 s
Wall time: 21.8 s


In [6]:
# load and add the new layers to the prev trained model
gmp_model = tf.keras.models.load_model('../../S_lab_TasselNet/models/tasselnet_global_max_pool_04_23_2022.h5')
# look at the input shape
gmp_model.input

2024-06-17 18:08:02.981537: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


<KerasTensor: shape=(None, None, None, 3) dtype=float32 (created by layer 'input_1')>

In [7]:
gmp_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                             

In [8]:
# need to remove some of the last layers
output_trial = gmp_model.layers[-7].output

In [9]:
# create the functional API model
reduced_model = tf.keras.models.Model(gmp_model.input, output_trial)
reduced_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                             

In [10]:
# okay, now need to add back the dropout, the dense and activation

# add dropout
added_dropout = tf.keras.layers.Dropout(0.5, name = "New_dropout_0")(gmp_model.layers[-7].output)

# add global max pooling
added_flatten = tf.keras.layers.GlobalMaxPooling2D()(added_dropout)

# add dense
added_dense = tf.keras.layers.Dense(64, name = "New_Dense_0")(added_flatten)

# add activation
added_Act = tf.keras.layers.Activation('relu', name = "New_Activation_0")(added_dense)

# add dropout
added_dropout2 = tf.keras.layers.Dropout(0.5, name = "New_dropout_1")(added_Act)

# add another dense
added_dense_1 = tf.keras.layers.Dense(32, name = "New_Dense_1")(added_dropout2)

# add activation
added_Act_1 = tf.keras.layers.Activation('relu', name = "New_Activation_1")(added_dense_1)

# add dropout
added_dropout3 = tf.keras.layers.Dropout(0.5, name = "New_dropout_2")(added_Act_1)

# add pred head
added_dense_3 = tf.keras.layers.Dense(1, name = "New_Dense_2")(added_dropout3)

# add activation
added_Act_2 = tf.keras.layers.Activation('relu', name = "New_Activation_2")(added_dense_3)

# define the new model with functional API
new_model = tf.keras.models.Model(gmp_model.input, added_Act_2)

In [11]:
# print the model summary
new_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                           

This is where we need to write the function - Rylie has two functions a generator, and also a batch-wise prediction function. May be we only need the generator function? Let's see.

In [12]:
def data_generator(x_data, y_data, batch_size, shuffle=False, peek=False, verbose=False):
    num_samples = len(x_data)
    indices = np.arange(num_samples)
    
    if peek:    # Give first batch unshuffled and don't change start index when peeking for training
        end = min(batch_size, num_samples)
        if verbose:
            print(f"Generating peeking batch up to index {end}")
        batch_x = x_data[:end]
        batch_y = y_data[:end]
        peek = False
        yield (batch_x, batch_y)

    while True:    # Loop indefinitely for epochs
        # Shuffle indices at the start of each epoch after the peek, if shuffle is enabled
        if shuffle:
            np.random.shuffle(indices)
        
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            batch_indices = indices[start:end]

            # Print batch indices if verbose
            if verbose:
                # Warning: calling verbose when shuffling will usually clutter output
                if shuffle:
                    if len(batch_indices) < 16:
                        print(f"Batch indices: {np.sort(batch_indices)}")
                    else:
                        print(f"Printing batch indices would clutter output. Skipped.")
                    print(f"Length: {len(batch_indices)}")
                else:
                    print(f"Generating batch from index {start} to {end}")
                # traceback.print_stack()

            # Generate batches
            batch_x = x_data[batch_indices]
            batch_y = y_data[batch_indices]

            # Yield the current batch
            yield (batch_x, batch_y)

In [13]:
%%time
batch_size = 1024
generator_batch_size = 1024
train_gen = data_generator(X_train, y_train, batch_size, shuffle=False, peek=True, verbose=False)
val_gen = data_generator(X_valid, y_valid, generator_batch_size, shuffle=False, peek=True, verbose=False)

CPU times: user 0 ns, sys: 4 µs, total: 4 µs
Wall time: 9.06 µs


In [14]:
steps_per_epoch = int(np.ceil(len(X_train) / batch_size))
train_steps = int(np.ceil(len(X_train) / generator_batch_size))
validation_steps = int(np.ceil(len(X_valid) / generator_batch_size))

In [15]:
steps_per_epoch, train_steps, validation_steps

(1536, 1536, 384)

In [16]:
%%time
# frozen model

#     freeze layers
for layer in gmp_model.layers:
    layer.trainable = False
    
# compile the model
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
new_model.compile(loss='mean_squared_error', optimizer=opt, metrics = ['mean_absolute_error'])
    
# add early stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights = True, verbose=1, patience=10)

# fit the generator - the steps are a must here
history = new_model.fit(train_gen,
          validation_data = val_gen, steps_per_epoch=steps_per_epoch, 
                                  validation_steps=validation_steps,                        
          epochs = 100, callbacks = [es])

Epoch 1/100


2024-06-17 18:08:05.677545: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-06-17 18:08:06.590921: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-06-17 18:08:14.781658: I external/local_xla/xla/service/service.cc:168] XLA service 0x144b812ada60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-17 18:08:14.781700: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100S-PCIE-32GB, Compute Capability 7.0
2024-06-17 18:08:14.940640: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1718665695.569995 2022668 device_compiler.h:186] Compiled cluster using XLA!  T

1536/1536 [==============================] - 39s 18ms/step - loss: 0.0301 - mean_absolute_error: 0.0679 - val_loss: 0.0209 - val_mean_absolute_error: 0.0457
Epoch 2/100
1536/1536 [==============================] - 28s 18ms/step - loss: 0.0279 - mean_absolute_error: 0.0696 - val_loss: 0.0204 - val_mean_absolute_error: 0.0539
Epoch 3/100
1536/1536 [==============================] - 27s 18ms/step - loss: 0.0272 - mean_absolute_error: 0.0689 - val_loss: 0.0203 - val_mean_absolute_error: 0.0562
Epoch 4/100
1536/1536 [==============================] - 28s 18ms/step - loss: 0.0269 - mean_absolute_error: 0.0681 - val_loss: 0.0201 - val_mean_absolute_error: 0.0565
Epoch 5/100
1536/1536 [==============================] - 28s 18ms/step - loss: 0.0267 - mean_absolute_error: 0.0674 - val_loss: 0.0200 - val_mean_absolute_error: 0.0559
Epoch 6/100
1536/1536 [==============================] - 27s 18ms/step - loss: 0.0265 - mean_absolute_error: 0.0670 - val_loss: 0.0200 - val_mean_absolute_error: 0.055

In [17]:
%%time
# # unfreeze few layers and retrain - finetune the model
gmp_model.trainable = True

set_trainable = False

for layer in gmp_model.layers:
    if layer.name == 'conv2d_3':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
            
    # # compile the mdoel
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
new_model.compile(loss='mean_squared_error', optimizer=opt, metrics = ['mean_absolute_error'])
    
    # # add early stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights = True, verbose=1, patience=10)
    
    # # fit the model (fine tuned)
history_new = new_model.fit(train_gen,
          validation_data = val_gen, steps_per_epoch=steps_per_epoch, 
                                  validation_steps=validation_steps,                        
          epochs = 100, callbacks = [es])


Epoch 1/100


2024-06-17 18:20:15.755560: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1536/1536 [==============================] - 32s 18ms/step - loss: 0.0250 - mean_absolute_error: 0.0610 - val_loss: 0.0196 - val_mean_absolute_error: 0.0498
Epoch 2/100
1536/1536 [==============================] - 28s 18ms/step - loss: 0.0243 - mean_absolute_error: 0.0595 - val_loss: 0.0194 - val_mean_absolute_error: 0.0516
Epoch 3/100
1536/1536 [==============================] - 28s 18ms/step - loss: 0.0239 - mean_absolute_error: 0.0586 - val_loss: 0.0192 - val_mean_absolute_error: 0.0532
Epoch 4/100
1536/1536 [==============================] - 28s 18ms/step - loss: 0.0236 - mean_absolute_error: 0.0577 - val_loss: 0.0190 - val_mean_absolute_error: 0.0532
Epoch 5/100
1536/1536 [==============================] - 28s 18ms/step - loss: 0.0234 - mean_absolute_error: 0.0572 - val_loss: 0.0188 - val_mean_absolute_error: 0.0526
Epoch 6/100
1536/1536 [==============================] - 28s 18ms/step - loss: 0.0232 - mean_absolute_error: 0.0567 - val_loss: 0.0187 - val_mean_absolute_error: 0.053

In [18]:
# save the finetuned model
model_name = 'trained_gmp_model_dense_32.h5'
new_model.save('models' + '/' + model_name)

/common/statsgeneral/gayara/tfp_for_TN/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
